In [1]:
%cd /home/ubuntu/shot-glass/python

import lunchbox.tools as lbt
from lunchbox.enforce import EnforceError
import pandas as pd

from shot_glass.core.monad import *
from shot_glass.core.df_monad import *

/home/ubuntu/shot-glass/python
WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


In [2]:
class Maybe(Monad):
    @classmethod
    def just(cls, value):
        return cls(value)
    
    @classmethod
    def nothing(cls):
        return cls(None)
    
    @property
    def state(self):
        if self._data is None:
            return 'nothing'
        return 'just'
    
    def __repr__(self):
        if self.state == 'just':
            return f'Just({self._data})'
        return 'Nothing'
    
    def _try(self, method, func):
        if self._data is None:
            return self.nothing()
        try:
            return method(func)
        except:
            return self.nothing()
    
    def bind(self, func):
        return self._try(super().bind, func)

    def fmap(self, func):
        return self._try(super().fmap, func)
    
class Either(Monad):
    @classmethod
    def success(cls, value):
        output = cls(value)
        output._constructor = 'success'
        return output
    
    @classmethod
    def failure(cls, error):
        output = cls(error)
        output._constructor = 'failure'
        return output
    
    def __repr__(self):
        return f'{self._constructor.capitalize()}({self._data})'
        
def try_(func, *args, **kwargs):
    def do(func, *args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            return e
    return Monad.wrap(do(func, *args, **kwargs))
    
try_(lambda x: x + '1', 1)
# try_(lambda x: x + '1', 'a')
print( Maybe.nothing().bind(lambda x: x / 0) )
print( Maybe.nothing().bind(lambda x: Either.success(x + 2)) )
print( Maybe.nothing().fmap(lambda x: x + 4) )
print( Maybe.just(1).bind(lambda x: x / 0) )
print( Maybe.just(1).bind(lambda x: Either.success(x + 2)) )
print( Maybe.just(1).fmap(lambda x: x + 4) )
# Either.success(1)
# Either.failure(4)

Maybe.just(1)

Nothing
Nothing
Nothing
Nothing
Success(3)
Just(5)


Just(1)

In [5]:
data = pd.DataFrame()
data['foo'] = [1, 2, 3]
data['bar'] = list('abc')
m = DFMonad(data) \
    .wrap_elments() \
    .fmap_elements(lambda x: x + 1)
m
# m = m.applymap(lambda x: x + 'x')
# m = m.applymap(lambda x: str(x) + 'x')
# m.unwrap()

TypeError: can only concatenate str (not "int") to str